# Scenario 2: A cross-functional team with one data scientis working on a ML model

Mflow setup:
###     - tracking server: yes, local server
###     - backend store: sqlite database
###     - artifacts store: local filesystem

To run this example you need to launch the mlflow server locally by running the following command in the terminal:
mlflow server --backend-store-uri sqlite:///backend.db

In [3]:
import mlflow

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()


[<Experiment: artifact_location='file:///C:/Users/mokon/Documents/Mlops-Zoomcamp/experiment_tracking/running_mlflow_examples/artifcts_local/0', creation_time=1747770218517, experiment_id='0', last_update_time=1747770218517, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("My-experiment-1")
with mlflow.start_run():

    X,y = load_iris(return_X_y=True)
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params)
    lr.fit(X,y)
    y_pred = lr.predict(X)
    mlflow.log_metrics({"accuracy": accuracy_score(y, y_pred)})
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print("default artifacts URI: {}".format(mlflow.get_artifact_uri()))

2025/05/20 13:45:32 INFO mlflow.tracking.fluent: Experiment with name 'My-experiment-1' does not exist. Creating a new experiment.
2025/05/20 13:45:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: file:///C:/Users/mokon/Documents/Mlops-Zoomcamp/experiment_tracking/running_mlflow_examples/artifcts_local/1/864f15818f2f4cbdbe6ea1c99684f768/artifacts
🏃 View run awesome-worm-676 at: http://127.0.0.1:5000/#/experiments/1/runs/864f15818f2f4cbdbe6ea1c99684f768
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///C:/Users/mokon/Documents/Mlops-Zoomcamp/experiment_tracking/running_mlflow_examples/artifcts_local/1', creation_time=1747770332674, experiment_id='1', last_update_time=1747770332674, lifecycle_stage='active', name='My-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/mokon/Documents/Mlops-Zoomcamp/experiment_tracking/running_mlflow_examples/artifcts_local/0', creation_time=1747770218517, experiment_id='0', last_update_time=1747770218517, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")


In [10]:
client.search_registered_models()

[]

#### We register one model

In [20]:
run = client.search_runs(experiment_ids=["1"])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run}/models",
    name = "iris classifier"
)

Registered model 'iris classifier' already exists. Creating a new version of this model...
2025/05/20 13:59:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris classifier, version 1
Created version '1' of model 'iris classifier'.


<ModelVersion: aliases=[], creation_timestamp=1747771199599, current_stage='None', description='', last_updated_timestamp=1747771199599, name='iris classifier', run_id='864f15818f2f4cbdbe6ea1c99684f768', run_link='', source='file:///C:/Users/mokon/Documents/Mlops-Zoomcamp/experiment_tracking/running_mlflow_examples/artifcts_local/1/864f15818f2f4cbdbe6ea1c99684f768/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>